In [1]:
# import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
import warnings
import sys
import os
warnings.filterwarnings("ignore")
sys.stderr = open(os.devnull, 'w')

# local crs
local_crs = 32636

In [2]:
region = gpd.read_file('data/leningrad_oblast_boundary.geojson', engine="pyogrio")
towns = gpd.read_file('data/towns.geojson', engine="pyogrio")
towns = towns.drop(columns=['population'])
towns = towns.rename(columns={'rounded_population': 'population'})
rayons = gpd.read_file('data/MO_LO.geojson', engine="pyogrio")
okrugs = gpd.read_file('data/GP_LO.geojson', engine="pyogrio")
adj_mx = pd.read_pickle('data/1_car_matrix.pickle')
adj_mx.index = towns.index
adj_mx.columns = towns.index

In [3]:
from popframe.preprocessing.level_filler import LevelFiller
level_filler = LevelFiller(towns=towns)
towns = level_filler.fill_levels()
towns

,geometry,name,population,level
0,POINT (33.75892 59.36226),Болото,10,Малое сельское поселение
1,POINT (33.78600 59.47517),Большой Остров,68,Малое сельское поселение
2,POINT (33.79236 59.47356),Бор,1734,Большое сельское поселение
3,POINT (33.77572 59.44249),Бороватое,10,Малое сельское поселение
4,POINT (33.67728 59.32819),Бочево,10,Малое сельское поселение
...,...,...,...,...
2925,POINT (31.23421 59.17021),Апраксин Бор,313,Среднее сельское поселение
2926,POINT (31.31924 59.18702),Александровка,313,Среднее сельское поселение
2927,POINT (31.47462 59.29408),Большая Горка,313,Среднее сельское поселение
2928,POINT (31.46265 59.30308),Дроздово,5,Малое сельское поселение


In [4]:
from popframe.method.popuation_frame import PopFrame
from popframe.models.region import Region

region = Region(
  region = region,
  towns=towns,
  settlements=okrugs, 
  districts=rayons, 
  accessibility_matrix=adj_mx,
)
